<a href="https://colab.research.google.com/github/MujkicAmar/Machine-Learning-For-CRAF-inhibitors/blob/main/CRAF_data_preprocessing_and_EDA(_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing necessary libraries

In [ ]:
! pip install rdkit

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.SaltRemover import SaltRemover
import numpy as np
from rdkit.Chem import Descriptors, Lipinski
from rdkit.Chem.FilterCatalog import *
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy
import math

# Searh for target CRAf protein in the CHEMBL database

In [ ]:
df = pd.read_csv("craf_doc.csv", index_col=[0])
df

# Handling missing data

In [ ]:
df2 = df[df.standard_value.notna()]
df2 = df[df.molecule_chembl_id.notna()]
df2 = df[df.canonical_smiles.notna()]
df2

#  Data pre-processing of the bioactivity data


In [ ]:
hand_selection_df = pd.read_csv("craf_no_db.csv")

In [ ]:
selection = ["molecule_chembl_id", "canonical_smiles", "standard_value"]
conc_data = df2[selection]
conc_data

In [ ]:
conc_data = conc_data.dropna()
conc_data

#Standardize

Adapted from: https://github.com/greglandrum/RSC_OpenScience_Standardization_202104/blob/main/Standardization%20and%20Validation%20with%20the%20RDKit.ipynb

In [ ]:
# Borrowed from https://bitsilla.com/blog/2021/06/standardizing-a-molecule-using-rdkit/
def standardize(smiles):
    # follows the steps in
    # https://github.com/greglandrum/RSC_OpenScience_Standardization_202104/blob/main/MolStandardize%20pieces.ipynb
    # as described **excellently** (by Greg) in
    # https://www.youtube.com/watch?v=eWTApNX8dJQ
    mol = Chem.MolFromSmiles(smiles)

    # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
    clean_mol = rdMolStandardize.Cleanup(mol)

    # if many fragments, get the "parent" (the actual mol we are interested in)
    parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)

    # try to neutralize molecule
    uncharger = rdMolStandardize.Uncharger()  # annoying, but necessary as no convenience method exists
    uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)

    # note that no attempt is made at reionization at this step
    # nor at ionization at some pH (rdkit has no pKa caculator)
    # the main aim to represent all molecules from different sources
    # in a (single) standard way, for use in ML, catalogue, etc.

    te = rdMolStandardize.TautomerEnumerator()  # idem
    taut_uncharged_parent_clean_mol = te.Canonicalize(uncharged_parent_clean_mol)

    return Chem.MolToSmiles(taut_uncharged_parent_clean_mol)


In [ ]:
smiles_df = conc_data.canonical_smiles.apply(standardize)
smiles_df

# Removing duplicates

Because data was taken from multiple scientific papers, it is possible that the different papers tested the same substance. We want to clean the data from any duplicates

In [ ]:
smiles_df = pd.DataFrame(smiles_df).rename(columns={"canonical_smiles": "std_smiles"})
conc_data2 = conc_data.drop(columns="canonical_smiles", axis=1)
final_data = pd.concat([conc_data2,smiles_df], axis=1)
final_data

In [ ]:
duplicates_droped_smi = final_data.drop_duplicates(subset="std_smiles", keep="first").reset_index().drop(columns="index", axis=1)
duplicates_droped_smi

# PAINS removal

In [ ]:
# initialize filter
params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
catalog = FilterCatalog(params)

# search for PAINS
matches = []
clean = []
for index, row in tqdm(duplicates_droped_smi.iterrows(), total=duplicates_droped_smi.shape[0]):
    molecule = Chem.MolFromSmiles(row.std_smiles)
    entry = catalog.GetFirstMatch(molecule)  # Get the first matching PAINS
    if entry is not None:
        # store PAINS information
        matches.append(
            {
                "chembl_id": row.molecule_chembl_id,
                "rdkit_molecule": molecule,
                "pains": entry.GetDescription().capitalize(),
            }
        )
    else:
        # collect indices of molecules without PAINS
        clean.append(index)

matches = pd.DataFrame(matches)
clean_data = duplicates_droped_smi.loc[clean]  # keep molecules without PAINS


In [ ]:
print(f"Number of compounds with PAINS: {len(matches)}")
print(f"Number of compounds without PAINS: {len(clean_data)}")

In [ ]:
clean_data = clean_data.reset_index(drop=True)

# Lipinski descriptors
Lipinski descriptors,  Lipinski's Rule-of-five, which proposes that molecules with poor permeation and oral absorption have molecular weight > 500, Clog P > 5, hydrogen-bond donor > 5 and hydrogen-bond acceptor > 10.

In [ ]:
#Code  from https://codeocean.com/explore/capsules?query=tag:data-curation

def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData= np.arange(1,1)
    i=0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        desc_numRotatingBonds = Descriptors.NumRotatableBonds(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors,
                        desc_numRotatingBonds])
        if (i == 0):
          baseData = row
        else:
          baseData = np.vstack([baseData, row])
        i=i+1

    columnNames=["MW","LogP","NumHDonors","NumHAcceptors", "numRotatingBonds"]
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)

    return descriptors

df_lipinski = generate(clean_data.std_smiles)
df_lipinski

In [ ]:
df_combined = pd.concat([clean_data, df_lipinski], axis=1)
df_combined

# Convert IC50 to pIC50

In [ ]:
def pIC50(i):
    molar = i*(10**-9)
    pIC50 = -np.log10(molar)
    return pIC50

In [ ]:
df_combined["norm_value"] = df_combined["standard_value"].astype(float).map(lambda x: 100000000.0  if x > 100000000.0 else x)
df_combined['pIC50'] = df_combined['norm_value'].apply(pIC50)
df_combined = df_combined.drop(columns="standard_value", axis=1)
df_combined


In [ ]:
bioactivity = df_combined.pIC50.astype(float)
bioactivity_class = bioactivity.map(lambda x: "inactive" if x <= 6 else "active")
bioactivity_class = pd.Series(bioactivity_class, name="bioactivity")
bioactivity_class

In [ ]:
exp_data = pd.concat([df_combined, bioactivity_class], axis=1)
exp_data

In [ ]:
exp_data.shape

In [ ]:
exp_data.dropna()

In [ ]:
exp_data.to_csv("CRAF_h.csv")

# Exploratory data analysis

In [ ]:
plt.figure(figsize=(8,8))


sns.countplot(x='bioactivity', data=exp_data)
plt.xlabel('bioactivity', size=20, fontweight='bold')
plt.ylabel('Frequency', fontsize=20, fontweight='bold')
plt.xticks(size=20)
plt.yticks(size=20)
sns.set(style='ticks')
sns.despine(top=True)
plt.tight_layout()
plt.savefig('plot_bioactivity_class.png')

In [ ]:
plt.figure(figsize=(8, 8))

sns.scatterplot(x='MW', y='LogP', data=exp_data, hue='bioactivity', size='pIC50', edgecolor='black', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

In [ ]:
selection = ['bioactivity','MW','LogP','NumHDonors','NumHAcceptors','pIC50', "numRotatingBonds"]
sns.pairplot(exp_data[selection], hue='bioactivity')
plt.tight_layout()
plt.savefig("Pairplot_of_selected data")

In [ ]:
exp_data

# Tests whether the Lipinski rule of five is fullfiled

In [ ]:
def fulfilled(df):
  predictions = []

  for i,row in df.iterrows():
        mw = row['MW']
        logp = row['LogP']
        hbd = row['NumHDonors']
        hba = row['NumHAcceptors']
        num_rot_b = row["numRotatingBonds"]

        # Check if all conditions are satisfied
        if mw <= 500 and logp <= 10 and hba <=10 and hbd <=5 and num_rot_b <=5:
            predictions.append(True)
        else:
            predictions.append(False)
  return pd.Series(predictions, name="fullfilment")

In [ ]:
ro5_properties = fulfilled(exp_data)

In [ ]:
fullfilment_test = pd.concat([exp_data, ro5_properties], axis=1)
fullfilment_test

In [ ]:
molecules_ro5_fulfilled = fullfilment_test[fullfilment_test.fullfilment == True]
molecules_ro5_violated = fullfilment_test[fullfilment_test.fullfilment == False]
len(molecules_ro5_fulfilled), len(molecules_ro5_violated)

#Making radar plot

In [ ]:
def calculate_mean_std(dataframe):
    stats = dataframe.describe()
    stats = stats.T
    stats = stats[["mean", "std"]]
    return stats

In [ ]:
molecules_ro5_fulfilled_stats = calculate_mean_std(molecules_ro5_fulfilled[['MW','LogP','NumHDonors','NumHAcceptors',"numRotatingBonds"]])
molecules_ro5_violated_stats = calculate_mean_std(molecules_ro5_violated[['MW','LogP','NumHDonors','NumHAcceptors',"numRotatingBonds"]])

In [ ]:
selections = ['MW','LogP','NumHDonors','NumHAcceptors',"numRotatingBonds"]
angles=np.linspace(0,2*np.pi,len(selections), endpoint=False)
angles=np.concatenate((angles,[angles[0]]))

In [ ]:
fig=plt.figure(figsize=(6,6))
ax=fig.add_subplot(polar=True)
ax.plot(angles)
plt.show()

In [ ]:
def _scale_by_thresholds(stats, thresholds, scaled_threshold):
    """
    Scale values for different properties that have each an individually defined threshold.

    Parameters
    ----------
    stats : pd.DataFrame
        Dataframe with "mean" and "std" (columns) for each physicochemical property (rows).
    thresholds : dict of str: int
        Thresholds defined for each property.
    scaled_threshold : int or float
        Scaled thresholds across all properties.

    Returns
    -------
    pd.DataFrame
        DataFrame with scaled means and standard deviations for each physiochemical property.
    """
    # Raise error if scaling keys and data_stats indicies are not matching
    for property_name in stats.index:
        if property_name not in thresholds.keys():
            raise KeyError(f"Add property '{property_name}' to scaling variable.")
    # Scale property data
    stats_scaled = stats.apply(lambda x: x / thresholds[x.name] * scaled_threshold, axis=1)
    return stats_scaled

In [ ]:
def _define_radial_axes_angles(n_axes):
    """Define angles (radians) for radial (x-)axes depending on the number of axes."""
    x_angles = [i / float(n_axes) * 2 * math.pi for i in range(n_axes)]
    x_angles += x_angles[:1]
    return x_angles

In [ ]:
#Code adapted from https://projects.volkamerlab.org/teachopencadd/talktorials/T002_compound_adme.html
def plot_radar(y, thresholds, scaled_threshold, properties_labels, y_max=None, output_path=None):

    x = _define_radial_axes_angles(len(y) - 1)
    y_scaled = _scale_by_thresholds(y, thresholds, scaled_threshold)

    # Set figure and subplot axis
    plt.figure(figsize=(6, 6))
    ax = plt.subplot(111, polar=True)

    # Plot data
    ax.fill(x, [scaled_threshold] * len(x), "cornflowerblue", alpha=0.2)
    ax.plot(x, y_scaled["mean"], "b", lw=3, ls="-")
    ax.plot(x, y_scaled["mean"] + y_scaled["std"], "orange", lw=2, ls="--")
    ax.plot(x, y_scaled["mean"] - y_scaled["std"], "orange", lw=2, ls="-.")

    # Set plot cosmetics
    ax.set_theta_offset(math.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(180)
    plt.xticks(x, [])
    if not y_max:
        y_max = int(ax.get_yticks()[-1])
    plt.ylim(0, y_max)
    plt.yticks(range(1, y_max), ["5" if i == scaled_threshold else "" for i in range(1, y_max)], fontsize=16)

    # Draw ytick labels
    for angle, label in zip(x, properties_labels):
        if angle == 0:
            ha = "center"
        elif 0 < angle < math.pi:
            ha = "left"
        elif angle == math.pi:
            ha = "center"
        else:
            ha = "right"
        ax.text(x=angle, y=y_max + 1, s=label, size=16, horizontalalignment=ha, verticalalignment="center")

    # Add legend
    labels = ("mean", "mean + std", "mean - std", "rule of five area")
    ax.legend(labels, loc=(1.1, 0.7), labelspacing=0.3, fontsize=16)

    # Save plot
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches="tight", transparent=True)

    plt.show()


In [ ]:
thresholds = {"MW": 500, "NumHAcceptors": 10, "NumHDonors": 5, "LogP": 5, "numRotatingBonds":5}
scaled_threshold = 5
properties_labels = [
    "Molecular weight (Da) / 100",
    "# NumHAcceptors / 2",
    "# NumHDonors",
    "LogP",
]
y_max = 8

Radar plot of molecules that fulfill Lipinski rule of five

In [ ]:
plot_radar(
    molecules_ro5_fulfilled_stats,
    thresholds,
    scaled_threshold,
    properties_labels,
    y_max,
)

Radar plot of molecules that violate the Lipinski rule of five

In [ ]:
plot_radar(
    molecules_ro5_violated_stats,
    thresholds,
    scaled_threshold,
    properties_labels,
    y_max,
)